In [1]:
import os
import geopandas as gpd
import time

from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import fiona
import itertools

In [2]:
scratch = os.path.join("..", "scratch")
shapefiles_loc = os.path.join("..", "data", "shapefiles")

In [3]:
bec_loc = os.path.join(shapefiles_loc, "bec_zones.shp")
bc_ppa = gpd.read_file(os.path.join(shapefiles_loc, "bc_ppa.shp"))
bc_boundary = gpd.read_file(os.path.join(shapefiles_loc, "bc_boundary.shp"))
bc_bounds_simple = gpd.read_file(os.path.join(shapefiles_loc, "bc_boundary_simplified.shp"))

In [9]:
bec = gpd.read_file(bec_loc)
bec = bec[['zone', 'subzone', 'geometry']]
start_time = time.time()
bec_dissolve = bec.dissolve(by = ['zone', 'subzone'], as_index = False)

dissolve_loc = os.path.join(scratch, "bec_subzones.shp")
bec_dissolve.to_file(os.path.join(dissolve_loc))
end_time = time.time()

print(end_time - start_time)

472.14177942276


In [12]:
subzones_loc = os.path.join(shapefiles_loc, "bec_subzones.shp")

print("exploding")
bec_exploded = bec_dissolve.explode()
bec_exploded.to_file(os.path.join(scratch, "bec_exploded.shp")) 

print("clipping")
bec_subzones_terr = gpd.clip(bec_exploded, bc_boundary)

print("saving clip")
bec_subzones_terr.to_file(subzones_loc)

exploding
clipping
saving clip


In [13]:
start_time = time.time()
print("unionizing") #lol
bc_ppa_bec_hres = gpd.overlay(bec_subzones_terr, bc_ppa, how = "union")
print(time.time() - start_time)
print("saving")
bc_ppa_bec_hres.to_file(os.path.join(shapefiles_loc, "bc_ppa_bec_hres.shp"))
end_time = time.time()

print(end_time - start_time)

67761.70734667778
